# Tensor visualization

## Libraries and Methods

In [1]:
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator

from ipywidgets import interact, fixed

from scipy import interpolate, integrate, special
from astropy.table import Table
import numpy as np

font = {'weight' : 'normal', 'size' : 30}
plt.rc('font', **font)

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

def _fz_DL(z):
    return cosmo.luminosity_distance(z).value

## Tensor Loading
Change the location of the tensor here if needed

In [2]:
class CR_Table:
    def __init__(self, filename, zmin, zmax):
        t = np.load(filename)
        self.logRi = t['logRi']
        self.z = t['z']
        self.logE = t['logE']
        self.ZA = t['ZA']
        self.tensor = t['tensor']

zmin, zmax = 0., 2.5
NuclName, Z, A = "C", 6, 12
filename = 'Output_Tensor/SimPropA_' + str(A) + '_z_' + str(zmax) + '.npz'
t = CR_Table(filename, zmin, zmax)

## Tensor content in $E_{\rm det}-E_{\rm inj}$ plane for varying injection distances and secondary species

In [3]:
def plotEE(t, NuclName, Z, A, iz_inj, jZA_det):

    # Compress the tensor along iz_inj
    weights = np.zeros_like(t.z)
    weights[iz_inj] = 1
    axz = np.where(np.array(t.tensor.shape) == t.z.size)
    tensor_stacked = np.tensordot(t.tensor, weights, axes=(axz[0],[0]))
    tensor_stacked[tensor_stacked==0]=np.nan

    # Select the tensor at jZA_det
    Einj = np.log10(Z)+t.logRi
    Edet = t.logE
    XX, YY = np.meshgrid(Einj, Edet)
    ZZ = np.transpose(tensor_stacked[jZA_det])

    # Plot the surface
    fig, ax = plt.subplots(figsize=(18, 15))
    surf = ax.pcolor(XX, YY, ZZ, cmap="inferno", vmin=0, vmax=A)
    ax.plot(Edet, Edet, color="k")
    plt.fill_between(x=Edet,y1=Edet,y2=np.max(Einj)*np.ones_like(Edet), facecolor="none", hatch="\\", edgecolor="k", linewidth=0.0)
    
    # Cosmetics
    title = r"$^{"+str(A)+r"}_{"+str(Z)+r"}$"+NuclName+" at "
    title +=  r"$D_L$ = " + str(np.round(_fz_DL(t.z[iz_inj] ),0)) + " Mpc, "
    title +=  r"$(Z_{\rm det},A_{\rm det})$ = "+str(t.ZA[jZA_det])
    ax.set_title(title)
    ax.set_xlabel(r"Injected log $E_{\rm inj}$/eV")
    ax.set_ylabel(r"Detected log $E_{\rm det}$/eV")
    ax.set_xlim(np.min(Edet), np.max(Edet))
    ax.set_ylim(np.min(Edet), np.max(Edet))
    
    # Add a color bar which maps values to colors.
    cbar = fig.colorbar(surf,fraction=0.046, pad=0.04)
    cbar.set_label("Mean # secondary nuclei")

interact(plotEE, t=fixed(t), NuclName = fixed(NuclName), Z=fixed(Z), A=fixed(A),  iz_inj=(0,t.z.size-1), jZA_det=(0,t.ZA.size-1));

interactive(children=(IntSlider(value=23, description='iz_inj', max=46), IntSlider(value=3, description='jZA_d…

## Tensor content as a function of injection distance for varying injected energies

In [5]:
def plotD(t, NuclName, Z, A, iE_inj, Nmin):

    # Compress the tensor along iE_inj
    weights = np.zeros_like(t.logRi)
    weights[iE_inj] = 1
    axRi = np.where(np.array(t.tensor.shape) == t.logRi.size)
    tensor_stacked = np.tensordot(t.tensor, weights, axes=(axRi[0],[0]))
    
    # Compress the tensor at all detected energies
    axEdet = np.where(np.array(tensor_stacked.shape) == t.logE.size)
    weights = np.ones_like(t.logE)
    tensor_stacked = np.tensordot(tensor_stacked, weights, axes=(axEdet[0],[0]))
            
    # Plot the curves
    fig, ax = plt.subplots(figsize=(18, 12))
    tot = 0
    for i in range(len(tensor_stacked)):
        if np.max(tensor_stacked[i])>Nmin:
            ax.plot(_fz_DL(t.z), tensor_stacked[i], label="(Z,A)="+str(t.ZA[i]),linewidth=2) 
        tot += tensor_stacked[i]
    ax.plot(_fz_DL(t.z), tot, label="Total detected", color='k',linewidth=2) 
    
    # Cosmetics
    logEinj = np.log10(Z)+t.logRi
    title = r"$^{"+str(A)+r"}_{"+str(Z)+r"}$"+NuclName+r" at log $E_{\rm inj}$/eV = " + str(np.round(logEinj[iE_inj],2))
    ax.set_title(title)    
    ax.set_xlabel(r"From luminosity distance, $D_L$ [Mpc]")
    ax.set_ylabel("Mean # nuclei" + r" at log $E_{\rm det}$/eV $\geq$ " + str(np.round(np.min(t.logE),2)))
    ax.set_xscale("log")
    ax.set_xlim(1, np.max(_fz_DL(t.z)))
    ax.set_ylim(0, np.max(tot)*1.1)
    
    ax.legend(loc="upper right")

logEinj = np.log10(Z)+t.logRi
iE_inj_min = np.nonzero(logEinj>=np.min(t.logE))[0][0]
iE_inj_max = np.nonzero(logEinj>np.max(t.logRi))[0][0]

interact(plotD, t=fixed(t), NuclName = fixed(NuclName), Z=fixed(Z), A=fixed(A),  iE_inj=(iE_inj_min,iE_inj_max), Nmin=fixed(0.2));

interactive(children=(IntSlider(value=19, description='iE_inj', max=31, min=8), Output()), _dom_classes=('widg…

## Tensor content as a function of injected energy for varying injection distances

In [6]:
def plotE(t, Z, A, iz_inj, Nmin):

    # Compress the tensor along iz_inj
    weights = np.zeros_like(t.z)
    weights[iz_inj] = 1
    axz = np.where(np.array(t.tensor.shape) == t.z.size) 
    tensor_stacked = np.tensordot(t.tensor, weights, axes=(axz[0],[0]))

    # Compress the tensor at all detected energies
    axEdet = np.where(np.array(tensor_stacked.shape) == t.logE.size)
    weights = np.ones_like(t.logE)
    tensor_stacked = np.tensordot(tensor_stacked, weights, axes=(axEdet[0],[0]))
    
    # Plot
    fig, ax = plt.subplots(figsize=(18, 12))
    logEinj = np.log10(Z)+t.logRi
    tot=0
    for i in range(len(tensor_stacked)):
        if np.max(tensor_stacked[i])>Nmin:
            ax.plot(logEinj, tensor_stacked[i], label="(Z,A)="+str(t.ZA[i]), linewidth=2) 
        tot += tensor_stacked[i]
    ax.plot(logEinj, tot, label="Total detected", color='k', linewidth=2, linestyle="--") 
    
    #Cosmetics
    title= title = r"$^{"+str(A)+r"}_{"+str(Z)+r"}$"+NuclName+r" at $D_L$ = " + str(np.round(_fz_DL(t.z[iz_inj]),0)) + " Mpc"
    ax.set_title(title)
    ax.set_xlabel(r"Injected log $E_{\rm inj}$/eV")
    ax.set_ylabel("Mean # nuclei" + r" at log $E_{\rm det}$/eV $\geq$ " + str(np.round(np.min(t.logE),2)))
    ax.set_xlim(np.min(t.logE), 20)
    ax.set_ylim(0, np.max(tot)*1.1)
    ax.set_yscale("symlog")
    
    ax.legend()

iz_min = np.nonzero(_fz_DL(t.z)>=1)[0][0]#>1 Mpc
iz_max = np.nonzero(_fz_DL(t.z)>1000)[0][0]#>1000 Mpc
interact(plotE, t=fixed(t), Z=fixed(Z), A=fixed(A),  iz_inj=(iz_min,iz_max), Nmin=fixed(0.2));

interactive(children=(IntSlider(value=17, description='iz_inj', max=33, min=2), Output()), _dom_classes=('widg…